In [ ]:
import time
import numpy as np
import pandas as pd
import scipy
import math
import random as rand
import pickle

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib import cm
from matplotlib import animation as animation
import matplotlib as mpl

from PIL import Image
from pylab import *

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split

from skimage.metrics import structural_similarity as ssim
from scipy.interpolate import griddata

from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Input, Conv2D, Concatenate
from keras.models import Model
import tensorflow as tf

from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

# DSOVT (COnvLSTM) - Basic Version

In [ ]:
vor_train = np.load('/content/drive/MyDrive/Physics/Physics/Dataset10/vor_train.npy', mmap_mode="r")[:30*300]
true_train = np.load('/content/drive/MyDrive/Physics/Physics/Dataset10/true_train.npy', mmap_mode="r")[:30*300]
vor_test = np.load('/content/drive/MyDrive/Physics/Physics/Dataset10/vor_test.npy', mmap_mode="r")
true_test = np.load('/content/drive/MyDrive/Physics/Physics/Dataset10/true_test.npy', mmap_mode="r")

In [ ]:
def custom_activation(x):
    # Adjust the slicing for 5D tensors
    tanh_activation = tf.keras.activations.tanh(x[:, :, :, :, :2])  # Apply tanh to the first two channels
    scaled_channel_3 = tf.keras.activations.relu(x[:, :, :, :, 2:3])  # Apply ReLU to the third channel, maintaining the last dimension
    return tf.concat([tanh_activation, scaled_channel_3], axis=-1)  # Concatenate along the channel dimension

# Define input shape
input_shape = (None, 5, 64, 64, 3)

# Input layer
inputs = layers.Input(shape=input_shape[1:])

# ConvLSTM layers
x = layers.ConvLSTM2D(
    filters=32,
    kernel_size=(3, 3),
    padding="same",
    return_sequences=True,
    activation="relu",
)(inputs)

x = layers.ConvLSTM2D(
    filters=32,
    kernel_size=(3, 3),
    padding="same",
    return_sequences=True,
    activation="relu",
)(x)


# Output layer
outputs = layers.Conv3D(
    filters=3,
    kernel_size=(3, 3, 3),
    activation=custom_activation,
    padding="same"
)(x)

# Create the model
model = models.Model(inputs, outputs)

# Compile the model
model.compile(loss=tf.keras.losses.mean_squared_error, optimizer=tf.keras.optimizers.Adam(), run_eagerly=True)

# Print model summary
model.summary()

In [ ]:
# Callback to save the best model
model_checkpoint = ModelCheckpoint(
    filepath='/content/drive/MyDrive/Physics/Physics/ConvLSTM-SW/best_model_convlstm_SW',#64_true.h5
    monitor='val_loss',
    save_best_only=True,
    verbose=1,
    mode='min',
    save_format='h5'
)


# Callbacks for adaptive learning rate, early stopping, and model checkpointing
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    patience=25,
    min_lr=1e-5
)
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=30,
    restore_best_weights=True
)

# Training the autoencoder
history = model.fit(
    x_convLSTM_train,
    y_convLSTM_train,
    epochs=100,
    batch_size=8,
    callbacks=[reduce_lr, model_checkpoint],  # Include model_checkpoint in the callbacks
    validation_data=(x_convLSTM_test, y_convLSTM_test),
    shuffle=True
)

In [ ]:
# Load the model with custom activation function
model = tf.keras.models.load_model('./Model_para/ConvLSTM/ConvLSTM-Origin-SW', custom_objects={'custom_activation': custom_activation})
start_time = time.time()
res_lstm = model.predict(x_convLSTM_test)

prediction_end_time = time.time()

print("Time taken for prediction:", prediction_end_time - start_time, "seconds")

In [ ]:
model = tf.keras.models.load_model('./Model_para/ConvLSTM/ConvLSTM-Origin-SW', custom_objects={'custom_activation': custom_activation})

# Predict using the model
start_time = time.time()
res_lstm = model.predict(x_convLSTM_test)
prediction_end_time = time.time()

print("Time taken for prediction:", prediction_end_time - start_time, "seconds")

In [ ]:
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

ssim_values = []
psnr_values = []

for i in range(len(y_convLSTM_test[:,0,:,:,:])):
    ssim_val = ssim(y_convLSTM_test[:,0,:,:,:][i], res_lstm[:,0,:,:,:][i], multichannel=True)
    psnr_val = psnr(y_convLSTM_test[:,0,:,:,:][i], res_lstm[:,0,:,:,:][i], data_range=2)

    ssim_values.append(ssim_val)
    psnr_values.append(psnr_val)

average_ssim = sum(ssim_values) / len(ssim_values)
average_psnr = sum(psnr_values) / len(psnr_values)

print(f"Average SSIM: {average_ssim}")
print(f"Average PSNR: {average_psnr}")
print(f"Average PSNR: {calculate_rrmse(y_convLSTM_test,res_lstm)}")

# DSOVT (COnvLSTM) - Physics Constrained Version

In [ ]:
vor_train = np.load('/content/drive/MyDrive/Physics/Physics/Dataset10/vor_train.npy', mmap_mode="r")[:10*300]
true_train = np.load('/content/drive/MyDrive/Physics/Physics/Dataset10/true_train.npy', mmap_mode="r")[:10*300]
vor_test = np.load('/content/drive/MyDrive/Physics/Physics/Dataset10/vor_test.npy', mmap_mode="r")
true_test = np.load('/content/drive/MyDrive/Physics/Physics/Dataset10/true_test.npy', mmap_mode="r")

## Physics-Constrained ConvLSTM

In [ ]:
def calculate_image_energy(img):
    g = 9.81
    dx = 1
    img = tf.reshape(img, (-1, img.shape[2], img.shape[3],3))

    kinetic_energy = 0.5 * tf.reduce_sum(tf.square(img[..., 0]), axis=(1, 2)) * dx**2
    kinetic_energy += 0.5 * tf.reduce_sum(tf.square(img[..., 1]), axis=(1, 2)) * dx**2
    # Calculate potential energy
    potential_energy = 0.5 * g * tf.reduce_sum(tf.square(img[..., 2]), axis=(1, 2)) * dx**2
    # Calculate total energy
    total_energy = kinetic_energy + potential_energy
    return total_energy

def calculate_mass_conservation(y):
    y = tf.reshape(y, (-1, y.shape[2], y.shape[3], 3))
    h, u, v = y[..., 2:3], y[..., 0:1], y[..., 1:2]
    dh_dx, dh_dy = tf.image.image_gradients(h)
    du_dx, _ = tf.image.image_gradients(u)
    _, dv_dy = tf.image.image_gradients(v)
    mass_conservation = dh_dx * u + h * du_dx + dh_dy * v + h * dv_dy

    mass_conservation_loss = tf.reduce_mean(tf.abs(mass_conservation), axis=[1, 2, 3])
    return mass_conservation_loss

def calculate_mass_conservation_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    mass_conservation_true = calculate_mass_conservation(y_true)
    mass_conservation_pred = calculate_mass_conservation(y_pred)
    return tf.reduce_mean(tf.abs(mass_conservation_pred - mass_conservation_true))

def calculate_relative_energy_error(x_corr_true, y_pred):
    energy_true = calculate_image_energy(x_corr_true)
    energy_pred = calculate_image_energy(y_pred)
    return tf.reduce_sum(tf.abs(energy_pred - energy_true))

In [ ]:
def prepare_convLSTM_data(x_data, y_data, num_of_sources, input_length=3, forecast_horizon=3):
    X, y = [], []

    # 计算每个源的样本数量
    samples_per_source = len(x_data) // num_of_sources

    for i in range(num_of_sources):
        # 根据每个源的样本数量计算起始和结束索引
        start_idx = i * samples_per_source
        end_idx = start_idx + samples_per_source

        # 切分x_data和y_data，获取当前源的数据
        x_source_data = x_data[start_idx:end_idx]
        y_source_data = y_data[start_idx:end_idx]

        # 将每个样本的源数量合并成一个新的维度
        x_sample = x_source_data
        y_sample = y_source_data

        # 将每个样本划分为输入和输出序列
        for j in range(samples_per_source - input_length - forecast_horizon + 1):
            X.append(x_sample[j:j+input_length])
            y.append(y_sample[j+input_length:j+input_length+forecast_horizon])

    return np.array(X), np.array(y)


input_length = 5
forecast_horizon = 5

# 计算源的数量
num_of_sources = len(vor_train) // 300
x_convLSTM_train, y_convLSTM_train = prepare_convLSTM_data(vor_train, true_train, num_of_sources, input_length, forecast_horizon)
x_convLSTM_test, y_convLSTM_test = prepare_convLSTM_data(vor_test, true_test, 10, input_length, forecast_horizon)
x_corr_true_train,_ = prepare_convLSTM_data(true_train,true_train, num_of_sources, input_length, forecast_horizon)
x_corr_true_test,_ = prepare_convLSTM_data(true_test,true_test, 10, input_length, forecast_horizon)

In [ ]:
def calculate_image_energy(img):
    g = tf.constant(9.81, dtype=tf.float32)  # Ensure g is float32
    dx = tf.constant(1, dtype=tf.float32)  # Ensure dx is float32
    img = tf.cast(img, tf.float32)  # Convert img to float32
    img = tf.reshape(img, (-1, img.shape[2], img.shape[3], 3))

    kinetic_energy = 0.5 * tf.reduce_sum(tf.square(img[..., 0]), axis=(1, 2)) * dx**2
    kinetic_energy += 0.5 * tf.reduce_sum(tf.square(img[..., 1]), axis=(1, 2)) * dx**2
    potential_energy = 0.5 * g * tf.reduce_sum(tf.square(img[..., 2]), axis=(1, 2)) * dx**2
    total_energy = kinetic_energy + potential_energy
    return total_energy

from tensorflow.keras import layers, models

def custom_activation(x):
    # Adjust the slicing for 5D tensors
    tanh_activation = tf.keras.activations.tanh(x[:, :, :, :, :2])  # Apply tanh to the first two channels
    scaled_channel_3 = tf.keras.activations.relu(x[:, :, :, :, 2:3])  # Apply ReLU to the third channel, maintaining the last dimension
    return tf.concat([tanh_activation, scaled_channel_3], axis=-1)  # Concatenate along the channel dimension

# Define input shape
input_shape = (None, 5, 64, 64, 3)

# Input layer
inputs = layers.Input(shape=input_shape[1:])

# ConvLSTM layers
x = layers.ConvLSTM2D(
    filters=32,
    kernel_size=(3, 3),
    padding="same",
    return_sequences=True,
    activation="relu",
)(inputs)

x = layers.ConvLSTM2D(
    filters=32,
    kernel_size=(3, 3),
    padding="same",
    return_sequences=True,
    activation="relu",
)(x)

# Output layer
outputs = layers.Conv3D(
    filters=3,
    kernel_size=(3, 3, 3),
    activation=custom_activation,
    padding="same"
)(x)

# Create the model
model = models.Model(inputs, outputs)

optimizer = tf.keras.optimizers.Adam()
batch_size = 32

train_dataset = tf.data.Dataset.from_tensor_slices((x_convLSTM_train, y_convLSTM_train, x_corr_true_train))
train_dataset = train_dataset.batch(batch_size)

val_dataset = tf.data.Dataset.from_tensor_slices((x_convLSTM_test, y_convLSTM_test, x_corr_true_test))
val_dataset = val_dataset.batch(batch_size)

# Define custom loss wrapper
def custom_loss_wrapper(input_tensor, epoch):
    def custom_loss(y_true, y_pred):
        if epoch < 15:
            # Use MSE loss for the first 20 epochs
            mse_loss = tf.keras.losses.mean_squared_error(y_true, y_pred)
            #energy_restriction = calculate_relative_energy_error(input_tensor, y_pred)

            total_loss = mse_loss #+ 1e-10 * energy_restriction
            return total_loss
        else:
            # Switch to custom loss after 20 epochs
            mse_loss = tf.keras.losses.mean_squared_error(y_true, y_pred)
            mass_conservation_loss_value = calculate_mass_conservation_loss(input_tensor, y_pred)
            energy_restriction = calculate_relative_energy_error(input_tensor, y_pred)
            total_loss = mse_loss + 5e-10 * energy_restriction#+0.0000001*mass_conservation_loss_value
            return total_loss

    return custom_loss

best_val_mse = float('inf')  # Initialize as infinity
best_model_path = './Model_para/ConvLSTM/ConvLSTM-Phy-SW'  # Model save path

# Training loop
for epoch in range(70):
    total_mse_loss_train = 0.0
    total_steps_train = 0
    total_mse_loss_val = 0.0
    total_steps_val = 0

    # Training
    for step, (x_batch, y_batch, x_corr_batch) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            y_pred = model(x_batch, training=True)
            # Calculate loss using custom loss wrapper
            loss_fn = custom_loss_wrapper(x_corr_batch, epoch)
            total_loss = loss_fn(y_batch, y_pred)
            total_mse_loss_train += tf.reduce_mean(total_loss).numpy()
            total_steps_train += 1
        grads = tape.gradient(total_loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

    average_mse_loss_train = total_mse_loss_train / total_steps_train
    print(f"Epoch {epoch+1}, Training Average MSE Loss: {average_mse_loss_train}")

    # Validation
    for x_batch, y_batch, x_corr_batch in val_dataset:
        y_pred = model(x_batch, training=False)
        # Calculate loss using custom loss wrapper
        loss_fn = custom_loss_wrapper(x_corr_batch, epoch)
        total_loss = loss_fn(y_batch, y_pred)
        total_mse_loss_val += tf.reduce_mean(total_loss).numpy()
        total_steps_val += 1

    average_mse_loss_val = total_mse_loss_val / total_steps_val
    print(f"Epoch {epoch+1}, Validation Average MSE Loss: {average_mse_loss_val}")

    # Check for best model
    if average_mse_loss_val < best_val_mse:
        best_val_mse = average_mse_loss_val
        model.save(best_model_path)  # Save the best model
        print(f"New best model saved at {best_model_path}")

### ConvLSTM - Origin

In [ ]:
import tensorflow as tf

def calculate_image_energy(img):
    g = tf.constant(9.81, dtype=tf.float32)  # Ensure g is float32
    dx = tf.constant(1, dtype=tf.float32)  # Ensure dx is float32
    img = tf.cast(img, tf.float32)  # Convert img to float32
    img = tf.reshape(img, (-1, img.shape[2], img.shape[3], 3))

    kinetic_energy = 0.5 * tf.reduce_sum(tf.square(img[..., 0]), axis=(1, 2)) * dx**2
    kinetic_energy += 0.5 * tf.reduce_sum(tf.squa kre(img[..., 1]), axis=(1, 2)) * dx**2
    potential_energy = 0.5 * g * tf.reduce_sum(tf.square(img[..., 2]), axis=(1, 2)) * dx**2
    total_energy = kinetic_energy + potential_energy
    return total_energy

import tensorflow as tf
from tensorflow.keras import layers, models

def custom_activation(x):
    # Adjust the slicing for 5D tensors
    tanh_activation = tf.keras.activations.tanh(x[:, :, :, :, :2])  # Apply tanh to the first two channels
    scaled_channel_3 = tf.keras.activations.relu(x[:, :, :, :, 2:3])  # Apply ReLU to the third channel, maintaining the last dimension
    return tf.concat([tanh_activation, scaled_channel_3], axis=-1)  # Concatenate along the channel dimension

# Define input shape
input_shape = (None, 5, 64, 64, 3)

# Input layer
inputs = layers.Input(shape=input_shape[1:])

# ConvLSTM layers
x = layers.ConvLSTM2D(
    filters=32,
    kernel_size=(3, 3),
    padding="same",
    return_sequences=True,
    activation="relu",
)(inputs)

x = layers.ConvLSTM2D(
    filters=32,
    kernel_size=(3, 3),
    padding="same",
    return_sequences=True,
    activation="relu",
)(x)

# Output layer
outputs = layers.Conv3D(
    filters=3,
    kernel_size=(3, 3, 3),
    activation=custom_activation,
    padding="same"
)(x)

# Create the model
orig_model = models.Model(inputs, outputs)

optimizer = tf.keras.optimizers.Adam()
batch_size = 32

train_dataset = tf.data.Dataset.from_tensor_slices((x_convLSTM_train, y_convLSTM_train, x_corr_true_train))
train_dataset = train_dataset.batch(batch_size)

val_dataset = tf.data.Dataset.from_tensor_slices((x_convLSTM_test, y_convLSTM_test, x_corr_true_test))
val_dataset = val_dataset.batch(batch_size)

# Define custom loss wrapper
def custom_loss_wrapper(input_tensor, epoch):
    def custom_loss(y_true, y_pred):
        if epoch < 100:
            # Use MSE loss for the first 20 epochs
            mse_loss = tf.keras.losses.mean_squared_error(y_true, y_pred)
            energy_restriction = calculate_relative_energy_error(input_tensor, y_pred)
            total_loss = mse_loss #+ 1e-10 * energy_restriction
            return total_loss
        else:
            # Switch to custom loss after 20 epochs
            mse_loss = tf.keras.losses.mean_squared_error(y_true, y_pred)
            energy_restriction = calculate_relative_energy_error(input_tensor, y_pred)
            total_loss = mse_loss + 6e-10 * energy_restriction
            return total_loss

    return custom_loss

best_val_mse = float('inf')  # Initialize as infinity
best_model_path = './Model_para/ConvLSTM/ConvLSTM-BP-SW'  # Model save path

from tensorflow.keras.callbacks import EarlyStopping

# Create the model
orig_model = models.Model(inputs, outputs)

# Define early stopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=25,          # Stop training if no improvement after 25 epochs
    restore_best_weights=True  # Restore weights to the best model
)

# Training loop
for epoch in range(50):
    total_mse_loss_train = 0.0
    total_steps_train = 0
    total_mse_loss_val = 0.0
    total_steps_val = 0

    # Training
    for step, (x_batch, y_batch, x_corr_batch) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            y_pred = orig_model(x_batch, training=True)
            # Calculate loss using custom loss wrapper
            loss_fn = custom_loss_wrapper(x_corr_batch, epoch)
            total_loss = loss_fn(y_batch, y_pred)
            total_mse_loss_train += tf.reduce_mean(total_loss).numpy()
            total_steps_train += 1
        grads = tape.gradient(total_loss, orig_model.trainable_variables)
        optimizer.apply_gradients(zip(grads, orig_model.trainable_variables))

    average_mse_loss_train = total_mse_loss_train / total_steps_train
    print(f"Epoch {epoch+1}, Training Average MSE Loss: {average_mse_loss_train}")

    # Validation
    for x_batch, y_batch, x_corr_batch in val_dataset:
        y_pred = orig_model(x_batch, training=False)
        # Calculate loss using custom loss wrapper
        loss_fn = custom_loss_wrapper(x_corr_batch, epoch)
        total_loss = loss_fn(y_batch, y_pred)
        total_mse_loss_val += tf.reduce_mean(total_loss).numpy()
        total_steps_val += 1

    average_mse_loss_val = total_mse_loss_val / total_steps_val
    print(f"Epoch {epoch+1}, Validation Average MSE Loss: {average_mse_loss_val}")

    # Check for best model
    if average_mse_loss_val < best_val_mse:
        best_val_mse = average_mse_loss_val
        orig_model.save(best_model_path)  # Save the best model
        print(f"New best model saved at {best_model_path}")

### Rolling Forecasting

In [ ]:
orig_model = tf.keras.models.load_model("./Model_para/ConvLSTM/ConvLSTM-BP-SW")
model = tf.keras.models.load_model("./Model_para/ConvLSTM/ConvLSTM-Physics-SW")

In [ ]:
# Load and preprocess data
reshaped_features = vor_test.reshape((10, 300, 64,64,3))

# Extract features for original model
extracted_features_batches = []
x = 5
for feature_batch in reshaped_features:
    extracted_batch = np.array([feature_batch[i:i+x] for i in range(0, feature_batch.shape[0] - 10, x)])
    extracted_features_batches.append(extracted_batch)
extracted_features_batches = np.array(extracted_features_batches)

# Extract true images for physics-based model
reshaped_features = true_test.reshape((10, 300, 64, 64, 3))
extracted_true_image = []
x = 5
for feature_batch in reshaped_features:
    extracted_batch = np.array([feature_batch[i:i+x] for i in range(10, feature_batch.shape[0], x)])
    extracted_true_image.append(extracted_batch)
extracted_true_image = np.array(extracted_true_image)

In [ ]:
def calculate_rrmse(true, predicted):
    # Calculate the Root Mean Square Error (RMSE)
    rmse = np.sqrt(np.mean((true - predicted) ** 2))

    # Calculate the mean of the true data
    mean_true = np.mean(true)

    # Avoid division by zero by handling the case where the mean of the true data is zero
    if mean_true == 0:
        return np.inf

    # Calculate the Relative Root Mean Square Error (R-RMSE)
    rrmse = rmse / mean_true
    return rrmse

In [ ]:

# Define starting position, step size, and sequence length
START_POSITION = 15
STEP_SIZE = 5


def rolling_forecast(model, initial_input, true_images):
    input_sequence = initial_input.reshape(1, STEP_SIZE, 64,64,3)  # Reshape to match LSTM input shape
    predictions = []
    mse_array = []
    true_images_array = []
    for i in range(32):  # Consider 5 steps at a time

        next_steps = model.predict(input_sequence)
        predicted_image = next_steps.reshape(-1, 64,64,3)

        true_image = true_images[START_POSITION + i]

        predictions.extend(predicted_image)
        mse = calculate_rrmse(true_image,predicted_image).reshape(-1, 1)#np.mean((predicted_image - true_image) ** 2, axis=(1, 2, 3)).reshape(-1, 1)

        mse_array.append(mse)
        true_images_array.append(true_image)
        input_sequence = next_steps
    return np.array(predictions), np.array(mse_array), np.array(true_images_array)

def prepare_data(features_batches, true_image_batches, batch_index):
    return features_batches[batch_index], true_image_batches[batch_index]

n = 1
features_batches, true_image_batches = extracted_features_batches[n:(n+9)], extracted_true_image[n:(n+9)]

all_mse_orig = []
all_mse_phy = []
pred_orig_list = []
pred_phy_list = []
hist_mse_orig = []
hist_mse_phy = []
true_orig_list = []  # Renamed from t_orig_list
true_phy_list = []

# Collect MSE for each data set
for i in range(9):
    x_test_cu, y_test_cu = prepare_data(features_batches, true_image_batches, i)
    initial_input = x_test_cu[START_POSITION, :, :]

    pred_orig, mse_orig, true_orig = rolling_forecast(orig_model, initial_input, y_test_cu)
    pred_phy, mse_phy, true_phy = rolling_forecast(model, initial_input, y_test_cu)

    avg_mse_per_group_orig = np.mean(mse_orig, axis=1)
    avg_mse_per_group_phy = np.mean(mse_phy, axis=1)

    hist_mse_orig.append(mse_orig)
    hist_mse_phy.append(mse_phy)
    all_mse_orig.append(avg_mse_per_group_orig)
    all_mse_phy.append(avg_mse_per_group_phy)

    pred_orig_list.append(pred_orig)
    pred_phy_list.append(pred_phy)
    true_orig_list.append(true_orig)
    true_phy_list.append(true_phy)

all_mse_orig = np.array(all_mse_orig)
all_mse_phy = np.array(all_mse_phy)

# Filter out groups with maximum MSE below the threshold
threshold = 1
is_below_threshold_orig = np.max(all_mse_orig, axis=(1, 2)) <= threshold
filtered_mse_orig = all_mse_orig[is_below_threshold_orig]

is_below_threshold_phy = np.max(all_mse_phy, axis=(1, 2)) <= threshold
filtered_mse_phy = all_mse_phy[is_below_threshold_phy]

# Calculate average and standard deviation of filtered MSE
avg_mse_orig = np.mean(filtered_mse_orig, axis=0)
std_mse_orig = np.std(filtered_mse_orig, axis=0)

avg_mse_phy = np.mean(filtered_mse_phy, axis=0)
std_mse_phy = np.std(filtered_mse_phy, axis=0)

# Flatten arrays for plotting
avg_mse_orig = np.asarray(avg_mse_orig).flatten()
std_mse_orig = np.asarray(std_mse_orig).flatten()
avg_mse_phy = np.asarray(avg_mse_phy).flatten()
std_mse_phy = np.asarray(std_mse_phy).flatten()

# Plot aggregated MSE with standard deviation
plt.figure(figsize=(10, 5))
plt.plot(avg_mse_orig, label='Original Model Average MSE')
plt.fill_between(range(len(avg_mse_orig)),
                 avg_mse_orig - std_mse_orig,
                 avg_mse_orig + std_mse_orig,
                 alpha=0.2)
plt.plot(avg_mse_phy, label='Physics-based Model Average MSE')
plt.fill_between(range(len(avg_mse_phy)), avg_mse_phy - std_mse_phy, avg_mse_phy + std_mse_phy, alpha=0.2)

plt.title('Aggregated Images MSE with Standard Deviation over Steps Compared to Decoded True Fields')
plt.xlabel('Step', fontsize=16)
plt.ylabel('MSE', fontsize=16)
plt.legend(loc='upper right', fontsize=9)
plt.grid(True)
plt.show()

In [ ]:
hist_mse_orig = np.array(hist_mse_orig).reshape((-1,1))
hist_mse_phy = np.array(hist_mse_phy).reshape((-1,1))
plt.figure(figsize=(15, 5))

color_orig_hist = 'skyblue'
color_phy_hist = 'orange'
edge_color = 'black'
plt.hist(hist_mse_orig, bins=30, alpha=0.75, color='skyblue', edgecolor='black', label='Basic ConvLSTM Model')
plt.hist(hist_mse_phy, bins=30, alpha=0.75, color='orange', edgecolor='black', label='Physics-based ConLSTM Model')

plt.xlabel('Error Value', fontsize=14)
plt.ylabel('Frequency', fontsize=14)
plt.title('Histogram of MSE Values for Two Models', fontsize=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(fontsize=12)

plt.grid(axis='y', linestyle='--', alpha=0.6)

plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.metrics import peak_signal_noise_ratio as psnr

# Define starting position, step size, and sequence length
START_POSITION = 15
STEP_SIZE = 5
SEQUENCE_LENGTH = 64

from skimage.metrics import peak_signal_noise_ratio as psnr

def rolling_forecast(model, initial_input, true_images):
    input_sequence = initial_input.reshape(1, STEP_SIZE, 64,64,3)  # Reshape to match LSTM input shape
    predictions = []
    psnr_array = []
    true_images_array = []
    for i in range(32):  # Consider 5 steps at a time
        next_steps = model.predict(input_sequence)
        predicted_image = next_steps.reshape(-1, 64,64,3)

        true_image = true_images[START_POSITION + i]

        # Convert TensorFlow tensor to NumPy array
        predicted_image_np = np.array(predicted_image)

        predictions.extend(predicted_image)
        psnr_val = psnr(true_image, predicted_image_np, data_range=predicted_image_np.max() - predicted_image_np.min())
        psnr_array.append([psnr_val])
        true_images_array.append(true_image)
        input_sequence = next_steps
    return np.array(predictions), np.array(psnr_array), np.array(true_images_array)


def prepare_data(features_batches, true_image_batches, batch_index):
    return features_batches[batch_index], true_image_batches[batch_index]

n = 1
features_batches, true_image_batches = extracted_features_batches[n:(n+9)], extracted_true_image[n:(n+9)]

all_psnr_orig = []
all_psnr_phy = []
pred_orig_list = []
pred_phy_list = []

true_orig_list = []  # Renamed from t_orig_list
true_phy_list = []  # Renamed from t_phy_list

# Collect PSNR for each data set
for i in range(9):
    x_test_cu, y_test_cu = prepare_data(features_batches, true_image_batches, i)
    initial_input = x_test_cu[START_POSITION, :, :]

    pred_orig, psnr_orig, true_orig = rolling_forecast(orig_model, initial_input, y_test_cu)
    pred_phy, psnr_phy, true_phy = rolling_forecast(model, initial_input, y_test_cu)

    avg_psnr_per_group_orig = np.mean(psnr_orig, axis=1)
    avg_psnr_per_group_phy = np.mean(psnr_phy, axis=1)

    all_psnr_orig.append(avg_psnr_per_group_orig)
    all_psnr_phy.append(avg_psnr_per_group_phy)

    pred_orig_list.append(pred_orig)
    pred_phy_list.append(pred_phy)
    true_orig_list.append(true_orig)
    true_phy_list.append(true_phy)

all_psnr_orig = np.array(all_psnr_orig)
all_psnr_phy = np.array(all_psnr_phy)

filtered_psnr_orig = all_psnr_orig
filtered_psnr_phy = all_psnr_phy

# Calculate average and standard deviation of filtered PSNR
avg_psnr_orig = np.mean(filtered_psnr_orig, axis=0)
std_psnr_orig = np.std(filtered_psnr_orig, axis=0)

avg_psnr_phy = np.mean(filtered_psnr_phy, axis=0)
std_psnr_phy = np.std(filtered_psnr_phy, axis=0)

# Flatten arrays for plotting
avg_psnr_orig = np.asarray(avg_psnr_orig).flatten()
std_psnr_orig = np.asarray(std_psnr_orig).flatten()
avg_psnr_phy = np.asarray(avg_psnr_phy).flatten()
std_psnr_phy = np.asarray(std_psnr_phy).flatten()

# Plot aggregated PSNR with standard deviation
plt.figure(figsize=(10, 5))
plt.plot(avg_psnr_orig, label='Original Model Average PSNR')
plt.fill_between(range(len(avg_psnr_orig)),
                 avg_psnr_orig - std_psnr_orig,
                 avg_psnr_orig + std_psnr_orig,
                 alpha=0.2)
plt.plot(avg_psnr_phy, label='Physics-based Model Average PSNR')
plt.fill_between(range(len(avg_psnr_phy)), avg_psnr_phy - std_psnr_phy, avg_psnr_phy + std_psnr_phy, alpha=0.2)

plt.title('Aggregated Images PSNR with Standard Deviation over Steps Compared to True Fields')
plt.xlabel('Step', fontsize=16)
plt.ylabel('PSNR', fontsize=16)
plt.legend(loc='upper right', fontsize=12)
plt.grid(True)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim

# Define starting position, step size, and sequence length
START_POSITION = 15
STEP_SIZE = 5
SEQUENCE_LENGTH = 64

def rolling_forecast(model, initial_input, true_images):
    input_sequence = initial_input.reshape(1, STEP_SIZE, 64,64,3)  # Reshape to match LSTM input shape
    predictions = []
    ssim_array = []
    true_images_array = []
    for i in range(32):  # Consider 5 steps at a time
        next_steps = model.predict(input_sequence)
        predicted_image = next_steps.reshape(-1, 64,64,3)
        true_image = true_images[START_POSITION + i]

        # Convert TensorFlow tensor to NumPy array
        predicted_image_np = np.array(predicted_image)

        predictions.extend(predicted_image)
        ssim_val, _ = ssim(predicted_image_np, true_image, multichannel=True, full=True, win_size=5)
        ssim_array.append([ssim_val])
        true_images_array.append(true_image)
        input_sequence = next_steps
    return np.array(predictions), np.array(ssim_array), np.array(true_images_array)

def prepare_data(features_batches, true_image_batches, batch_index):
    return features_batches[batch_index], true_image_batches[batch_index]

n = 1
features_batches, true_image_batches = extracted_features_batches[n:(n+9)], extracted_true_image[n:(n+9)]

all_ssim_orig = []
all_ssim_phy = []
pred_orig_list = []
pred_phy_list = []

true_orig_list = []  # Renamed from t_orig_list
true_phy_list = []  # Renamed from t_phy_list

# Collect SSIM for each data set
for i in range(9):
    x_test_cu, y_test_cu = prepare_data(features_batches, true_image_batches, i)
    initial_input = x_test_cu[START_POSITION, :, :]

    pred_orig, ssim_orig, true_orig = rolling_forecast(orig_model, initial_input, y_test_cu)
    pred_phy, ssim_phy, true_phy = rolling_forecast(model, initial_input, y_test_cu)

    avg_ssim_per_group_orig = np.mean(ssim_orig, axis=1)
    avg_ssim_per_group_phy = np.mean(ssim_phy, axis=1)

    all_ssim_orig.append(avg_ssim_per_group_orig)
    all_ssim_phy.append(avg_ssim_per_group_phy)

    pred_orig_list.append(pred_orig)
    pred_phy_list.append(pred_phy)
    true_orig_list.append(true_orig)
    true_phy_list.append(true_phy)

In [ ]:
plt.figure(figsize=(15, 5))

# Define color palette
color_orig = 'blue'
color_phy = 'red'
color_fill_orig = 'lightblue'
color_fill_phy = 'salmon'  # Using 'salmon' as a lighter shade of red

# Plot MSE
plt.subplot(1, 3, 1)
plt.plot(avg_mse_orig, label='Basic Model', color=color_orig)
plt.fill_between(range(len(avg_mse_orig)),
                 avg_mse_orig - std_mse_orig,
                 avg_mse_orig + std_mse_orig,
                 alpha=0.2, color=color_fill_orig)
plt.plot(avg_mse_phy, label='Physics-based Model', color=color_phy)
plt.fill_between(range(len(avg_mse_phy)),
                 avg_mse_phy - std_mse_phy,
                 avg_mse_phy + std_mse_phy,
                 alpha=0.2, color=color_fill_phy)
plt.title('R-RMSE with Standard Deviation over Steps', fontsize=16)
plt.xlabel('Iteration', fontsize=12)
plt.ylabel('R-RMSE', fontsize=12)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(loc='upper left', fontsize=10)
plt.grid(True)

# Plot SSIM
plt.subplot(1, 3, 2)
plt.plot(avg_ssim_orig, label='Basic Model', color=color_orig)
plt.fill_between(range(len(avg_ssim_orig)),
                 avg_ssim_orig - std_ssim_orig,
                 avg_ssim_orig + std_ssim_orig,
                 alpha=0.2, color=color_fill_orig)
plt.plot(avg_ssim_phy, label='Physics-based Model', color=color_phy)
plt.fill_between(range(len(avg_ssim_phy)),
                 avg_ssim_phy - std_ssim_phy,
                 avg_ssim_phy + std_ssim_phy,
                 alpha=0.2, color=color_fill_phy)
plt.title('SSIM with Standard Deviation over Steps', fontsize=16)
plt.xlabel('Iteration', fontsize=12)
plt.ylabel('SSIM', fontsize=12)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(loc='upper right', fontsize=10)
plt.grid(True)

# Plot PSNR
plt.subplot(1, 3, 3)
plt.plot(avg_psnr_orig, label='Basic Model', color=color_orig)
plt.fill_between(range(len(avg_psnr_orig)),
                 avg_psnr_orig - std_psnr_orig,
                 avg_psnr_orig + std_psnr_orig,
                 alpha=0.2, color=color_fill_orig)
plt.plot(avg_psnr_phy, label='Physics-based Model', color=color_phy)
plt.fill_between(range(len(avg_psnr_phy)),
                 avg_psnr_phy - std_psnr_phy,
                 avg_psnr_phy + std_psnr_phy,
                 alpha=0.2, color=color_fill_phy)
plt.title('PSNR with Standard Deviation over Steps', fontsize=16)
plt.xlabel('Iteration', fontsize=12)
plt.ylabel('PSNR', fontsize=12)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(loc='upper right', fontsize=10)
plt.grid(True)

plt.tight_layout()
plt.show()